# SNMP - Simple Network Management Protocol

## Install snmpd tools, agent and mibs

In [ ]:
%%bash

# Install SNMP - Linux
sudo apt update
sudo apt install -y snmp snmpd snmp-mibs-downloader

## Configure agent configuration file (/etc/snmp/snmpd.conf) and snmp tools configuration file (/etc/snmp/snmp.conf)

In [2]:
%%bash

sudo mv /etc/snmp/snmpd.conf /etc/snmp/snmpd.conf.old

sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private
EOF

sudo sed -i '/mibs :/s/^/#/' /etc/snmp/snmp.conf

## Start snmpd daemon (agent)

In [ ]:
%%bash

sudo service snmpd restart
sudo service snmpd status

# Mac
# sudo launchctl unload /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist
# sudo launchctl load -w /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist

## List all downloaded mibs

In [ ]:
%%bash

ls -R /usr/share/snmp/mibs

## Show RFC1213 object tree

In [ ]:
%%bash

# Variables
MIB="/usr/share/snmp/mibs/ietf/RFC1213-MIB"

snmptranslate -m $MIB -Tp

## Show the description of a specific object

In [ ]:
%%bash

# Variables
OBJECT="sysDescr"
MIB="/usr/share/snmp/mibs/ietf/RFC1213-MIB"

# Command to extract text from search_string to } excluding the }
sed -n "/$OBJECT OBJECT-TYPE/,/}/p" $MIB

## Using snmpget

In [ ]:
%%bash

# snmpget -v <SNMP version> -c <community> <host> <obj1> <obj2> <obj3> 

snmpget -v 1 -c public localhost sysDescr.0 sysContact.0

## Using snmpset

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysContact.0 s admin@localhost
snmpget -v 1 -c public localhost sysContact.0

## Using snmpgetnext

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost system interfaces

## Using snmpwalk

In [ ]:
%%bash

# run on terminal: sudo tcpdump -i lo -n port 161 -v
snmpwalk -v 1 -c public localhost system

## Other commands

In [ ]:
%%bash

snmptable -v 1 -c public localhost ifTable

In [ ]:
%%bash

snmpdelta -v 1 -c public -Cs -CT localhost sysUpTime.0

In [ ]:
%%bash

killall snmpdelta

## Exercícios - SNMPv1

a) obter o endereço físico (MAC) da 2a. interface de rede da tabela ifTable

In [ ]:
%%bash

snmpget -v 1 -c public localhost ifPhysAddress.2

b) obter o número de mensagens ICMP enviadas e recebidas pelo elemento gerenciado

In [ ]:
%%bash

snmpget -v 1 -c public localhost icmpInMsgs.0 icmpOutMsgs.0

c) obter o número de requisições GET recebidos pelo agente no elemento gerenciado.

In [ ]:
%%bash

snmpget -v 1 -c public localhost snmpInGetRequests.0

d) modificar o nome do elemento gerenciado para "Gerencia"

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysName.0 s "Gerencia"

e) modificar a identificação da localização do elemento gerenciado para "LabRedes"

In [ ]:
%%bash

snmpset -v 1 -c private localhost sysLocation.0 s "LabRedes"

f) obter a descrição da interface localizada na 1a. linha da tabela ifTable

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost ifDescr

g) obter o endereço IP, índice da interface na tabela ifTable, máscara de rede e endereço de broadcast da primeira entrada da tabela ipAddrEntry

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost ipAdEntIfIndex ipAdEntNetMask ipAdEntBcastAddr

h) obter o endereço local e porta local da primeira linha da tabela udpTable

In [ ]:
%%bash

snmpgetnext -v 1 -c public localhost udpLocalPort udpLocalAddress

## Using snmpbulkget

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn2 -Cr3 system interfaces ifIndex ifDescr ifSpeed

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn4 -Cr5 tcpInSegs tcpOutSegs tcpRetransSegs tcpInErrs tcpConnState tcpConnLocalAddress tcpConnLocalPort tcpConnRemAddress tcpConnRemPort

In [ ]:
%%bash

snmpbulkget -v2c -c public localhost -Cn0 -Cr4 udpLocalAddress udpLocalPort

## Comparing snmpwalk vs snmpbulkwalk



In [ ]:
%%bash

# run on terminal: sudo tcpdump -i lo -n port 161 -v
# snmpwalk -v 1 -c public localhost system

snmpbulkwalk -v 2c -c public localhost system

# Example using pass directive

## Rewrite snmpd.conf and restart agent

In [ ]:
%%bash

# Add pass directive to snmpd.conf
sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private

pass .1.3.6.1.3.1234.1 /usr/bin/python3 /tmp/agent.py
EOF

# Restart agent - Linux
sudo service snmpd restart
sudo service snmpd status

# Restart agent - Mac
# sudo launchctl unload /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist
# sudo launchctl load -w /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist

## Agent code

In [ ]:
%%writefile /tmp/agent.py

import sys
import datetime
import socket
import os

def get_current_datetime():
    now = datetime.datetime.now()
    return now.strftime("%Y-%m-%d %H:%M:%S")

def get_hostname():
    hostname = socket.gethostname()
    return hostname

def get_file_content():
    filepath = '/tmp/myfile.txt'
    if not os.path.exists(filepath):
        with open(filepath, 'w') as f:
            f.write("initial")
        return "initial"
    with open(filepath, 'r') as f:
        return f.read()

def set_file_content(new_content):
    filepath = '/tmp/myfile.txt'
    with open(filepath, 'w') as f:
        f.write(new_content)
    return new_content

def main():

    with open("/tmp/agent.log", 'a') as file:
        file.write(' '.join(sys.argv) + '\n')

    if len(sys.argv) < 3:
        print("Usage: agent.py <request-type> <MIB-oid> [type] [<new-value>]")
        return

    request_type = sys.argv[1]
    oid = sys.argv[2]

    if request_type == "-g":  # GET request
        if oid == ".1.3.6.1.3.1234.1.1.0":
            print(".1.3.6.1.3.1234.1.1.0")
            print("string")
            print(get_current_datetime())
        elif oid == ".1.3.6.1.3.1234.1.2.0":
            print(".1.3.6.1.3.1234.1.2.0")
            print("string")
            print(get_hostname())
        elif oid == ".1.3.6.1.3.1234.1.3.0":
            print(".1.3.6.1.3.1234.1.3.0")
            print("string")
            print(get_file_content())
        else:
            print("NONE")
    elif request_type == "-s":  # SET request
        if oid == ".1.3.6.1.3.1234.1.3.0" and len(sys.argv) == 5:
            new_content = sys.argv[4]
            print(".1.3.6.1.3.1234.1.3.0")
            print("string")
            print(set_file_content(new_content))
        else:
            print("NONE")
    else:
        print("NONE")

if __name__ == "__main__":
    main()

## Testing get and set

In [ ]:
%%bash

snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.1.0
# snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.2.0
# snmpget -v2c -c public localhost .1.3.6.1.3.1234.1.3.0

In [ ]:
%%bash

snmpset -v2c -c private localhost .1.3.6.1.3.1234.1.3.0 s teste

## Write MIB file

In [ ]:
%%writefile MYMIB.txt

MYMIB DEFINITIONS ::= BEGIN

IMPORTS
    OBJECT-GROUP FROM SNMPv2-CONF
    OBJECT-TYPE, experimental FROM SNMPv2-SMI
    DisplayString FROM SNMPv2-TC;

myMIB MODULE-IDENTITY
    LAST-UPDATED "202406290000Z"
    ORGANIZATION "PUCRS"
    CONTACT-INFO "admin@localhost"
    DESCRIPTION "My MIB"
::= { experimental 1234 }

myObjects OBJECT IDENTIFIER ::= { myMIB 1 }

myCurrentDateTime OBJECT-TYPE
    SYNTAX DisplayString
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION "Current Date and Time"
    ::= { myObjects 1 }

myHostname OBJECT-TYPE
    SYNTAX DisplayString
    MAX-ACCESS read-only
    STATUS current
    DESCRIPTION "Hostname"
    ::= { myObjects 2 }

myFileContent OBJECT-TYPE
    SYNTAX DisplayString
    MAX-ACCESS read-write
    STATUS current
    DESCRIPTION "File Content"
    ::= { myObjects 3 }

END

## Get and set using the MIB file

In [ ]:
%%bash

snmpget -v2c -c public -M +. -m +MYMIB localhost myCurrentDateTime.0
snmpget -v2c -c public -M +. -m +MYMIB localhost myHostname.0
snmpget -v2c -c public -M +. -m +MYMIB localhost myFileContent.0


In [ ]:
%%bash

snmpset -v2c -c private -M +. -m +MYMIB localhost myFileContent.0 s teste

# Example using pass_ persist


## Install scapy and configure python to capture packets

In [ ]:
%%bash

# Install scapy on local python3 installation (since we'll be using /usr/bin/python3 to call the agent inside snmpd.conf)
sudo /usr/bin/pip3 install scapy

# Give permission for /usr/bin/python3 to capture network packets
sudo setcap cap_net_raw,cap_net_admin=eip $(realpath $(which /usr/bin/python3))

## Rewrite snmpd.conf and restart agent

In [ ]:
%%bash

# Add pass directive to snmpd.conf
sudo tee /etc/snmp/snmpd.conf > /dev/null <<EOF
rocommunity public
rwcommunity private

pass_persist .1.3.6.1.2.1.16.1 /usr/bin/python3 /tmp/probe.py
EOF

# Restart agent - Linux
sudo service snmpd restart
sudo service snmpd status

# Restart agent - Mac
# sudo launchctl unload /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist
# sudo launchctl load -w /System/Library/LaunchDaemons/org.net-snmp.snmpd.plist

## Probe code

In [ ]:
%%writefile /tmp/probe.py

#!/usr/bin/python3

import threading
from scapy.all import sniff
import time
import sys
import datetime
import socket
import os

# Shared variable to store the packet count
packet_count = 0
# Lock for thread-safe access to shared variable
count_lock = threading.Lock()

# Function to handle each sniffed packet
def packet_handler(packet):
    global packet_count
    with count_lock:
        packet_count += 1

# Sniffer thread function
def packet_sniffer():
    sniff(prn=packet_handler,iface="eth0", store=False, promisc=True)

def main():

    while True:

        line = sys.stdin.readline()
        if not line:
            raise EOFError()
        line = line.strip()

        if 'PING' in line:
            print("PONG")

            # Create and start the sniffer thread
            sniffer_thread = threading.Thread(target=packet_sniffer)
            sniffer_thread.daemon = True  # This makes the thread exit when the main program exits
            sniffer_thread.start()

        elif 'get' in line:
            oid = sys.stdin.readline()
            oid = oid.strip()
            if oid == ".1.3.6.1.2.1.16.1.1.1.1.1":
                print(".1.3.6.1.2.1.16.1.1.1.1.1")
                print("integer")
                with count_lock:
                    print(f"{packet_count}")
            else:
                print("NONE")
        else:
            print("NONE")

        sys.stdout.flush()

if __name__ == "__main__":
    main()

In [ ]:
%%bash

snmpget -v2c -c public localhost .1.3.6.1.2.1.16.1.1.1.1.1